In [3]:
import json
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from scipy.special import stirling2
import math
from copy import copy

In [4]:
status_codes = {
    1: 'LOADED',
    2: 'OPTIMAL',
    3: 'INFEASIBLE',
    4: 'INF_OR_UNBD',
    5: 'UNBOUNDED',
    6: 'CUTOFF',
    7: 'ITERATION_LIMIT',
    8: 'NODE_LIMIT',
    9: 'TIME_LIMIT',
    10: 'SOLUTION_LIMIT',
    11: 'INTERRUPTED',
    12: 'NUMERIC',
    13: 'SUBOPTIMAL',
    14: 'INPROGRESS',
    15: 'USER_OBJ_LIMIT'
}

In [5]:
rng = np.random.default_rng(8)

# SDP Moment Equations for Independent Telegraph

Adapt 2 species setup from 'interacting_birth_death' to 1 observed, 1 unobserved species for independent telegraph model

## Code

In [6]:
def compute_order(alpha):
    order = 0
    for alpha_i in alpha:
        order += alpha_i
    return order

In [7]:
def compute_Nd(d, S=2):
    '''Number of moments of order <= d (S species)'''
    Nd = math.factorial(S + d) // (math.factorial(d) * math.factorial(S))
    return Nd

In [8]:
def compute_powers(d, S=2):
    '''Compute the Nd powers of order <= d (S species)'''

    # all powers
    powers = [[0 for s in range(S)]]

    # powers of order d = 0
    powers_prev = [[0 for s in range(S)]]

    # for order d = 1, ..., d
    for order in range(1, d + 1):

        # store powers of order d
        powers_current = []

        # for each power of order d - 1
        for alpha in powers_prev:

            # for each index
            for i in range(S):

                # add 1 to power at index
                alpha_new = copy(alpha)
                alpha_new[i] += 1

                # store (avoid repeats)
                if alpha_new not in powers_current:
                    powers_current.append(alpha_new)

        # update d - 1 powers
        powers += powers_current

        # update overall powers
        powers_prev = powers_current

    return powers

In [9]:
def compute_A(alpha, d):
    '''
    Moment equation coefficient matrix
    NOTE: must have order of alpha <= d

    Args:
        alpha: moment order for equation (d/dt mu^alpha = 0)
        d: maximum moment order used

    Returns:
        A: (4, Nd) matrix of coefficients
    '''

    if compute_order(alpha) > d:
        raise NotImplementedError("Maximum moment order 'd' too small for order 'alpha' moment equation: involves moments of order higher than d.")

    # reaction settings
    R = 4
    S = 2

    xs = sp.symbols([f'x{i}' for i in range(S)])

    # reaction propensity polynomials
    props = [
        1 - xs[1],
        xs[1],
        xs[1],
        xs[0]
    ]

    # reaction vectors
    vrs = [
        [0, 1],
        [0, -1],
        [1, 0],
        [-1, 0]
    ]

    # number of moments of order <= d
    Nd = compute_Nd(d, S)

    # get powers of order <= d
    powers = compute_powers(d, S)

    # setup matrix
    A = np.zeros((R, Nd))

    for r, prop in enumerate(props):

        # expand b(x) * ((x + v_r)**alpha - x**alpha)
        term_1 = 1
        term_2 = 1
        for i in range(S):
            term_1 = term_1 * (xs[i] + vrs[r][i])**alpha[i]
            term_2 = term_2 * xs[i]**alpha[i]
        poly = sp.Poly(prop * (term_1 - term_2), xs)

        # loop over terms
        for xs_power, coeff in zip(poly.monoms(), poly.coeffs()):

            # get matrix index
            col = powers.index(list(xs_power))

            # store
            A[r, col] = coeff

    return A

## B Scaling matrix

Provide set of unobserved species $U$ which are not affected by capture efficiency

In [10]:
def compute_B(beta, d, U, S=2):
    '''
    Capture efficiency moment scaling matrix

    Args:
        beta: per cell capture efficiency sample
        d: maximum moment order used
        U: unobserved species indices
        S: number of species

    Returns:
        B: (Nd, Nd) matrix of coefficients
    '''

    # number of moments of order <= d
    Nd = compute_Nd(d, S)

    # compute powers of order <= d
    powers = compute_powers(d, S)

    # compute beta moments of order <= d
    y_beta = np.zeros(d + 1)
    for l in range(d + 1):
        y_beta[l] = np.mean(beta**l)

    def falling_factorial(n, k):
        val = 1
        for i in range(k):
            val *= (n - i)
        return val

    def binomial_moment(n, p, l):
        val = 0
        for k in range(l + 1):
            val += falling_factorial(n, k) * stirling2(l, k) * p**k
        return val

    # setup matrix
    B = np.zeros((Nd, Nd))

    p = sp.Symbol('p')
    xs = sp.symbols([f'x{i}' for i in range(S)])

    # for each moment power
    for row, alpha in enumerate(powers):

        # setup polynomail
        poly_alpha = 1

        # for each species
        for i in range(S):

            # unobserved: no capture efficiency
            if i in U:
                moment = xs[i]**alpha[i]

            # observed: compute moment expression for E[Xi^alphai] in xi
            else:
                moment = binomial_moment(xs[i], p, alpha[i])
            
            poly = sp.Poly(moment, p, xs[i])

            # multiply
            poly_alpha = poly_alpha * poly

        # loop over terms
        for (beta_power, *xs_power), coeff in zip(poly_alpha.monoms(), poly_alpha.coeffs()):

            # get matrix index
            col = powers.index(xs_power)

            B[row, col] += coeff * y_beta[beta_power]

    return B

In [11]:
def add_powers(*powers, S):
    plus = [0 for i in range(S)]
    for i in range(S):
        for power in powers:
            plus[i] += power[i]
    return plus

In [12]:
def construct_M_0(y, d, S):
    '''Moment matrix variable constructor (s = 0).'''
    D = math.floor(d / 2)
    powers_D = compute_powers(D, S)
    powers_d = compute_powers(d, S)
    ND = compute_Nd(D, S)
    M_0 = [[0 for j in range(ND)] for i in range(ND)]
    for alpha_index, alpha in enumerate(powers_D):
        for beta_index, beta in enumerate(powers_D):
            plus = add_powers(alpha, beta, S=2)
            plus_index = powers_d.index(plus)
            M_0[alpha_index][beta_index] = y[plus_index].item()
    M_0 = gp.MVar.fromlist(M_0)
    return M_0

In [13]:
def construct_M_s(y, d, s, S):
    '''Moment matrix variable constructor (s).'''
    D = math.floor((d - 1) / 2)
    powers_D = compute_powers(D, S)
    powers_d = compute_powers(d, S)
    ND = compute_Nd(D, S)
    M_s = [[0 for j in range(ND)] for i in range(ND)]
    e_s = [1 if i == (s - 1) else 0 for i in range(S)]
    for alpha_index, alpha in enumerate(powers_D):
        for beta_index, beta in enumerate(powers_D):
            plus = add_powers(alpha, beta, e_s, S=2)
            plus_index = powers_d.index(plus)
            M_s[alpha_index][beta_index] = y[plus_index].item()
    M_s = gp.MVar.fromlist(M_s)
    return M_s

In [31]:
def base_model(model, OB_bounds, beta, d, U, time_limit=300):
    '''
    Construct 'base model' with semidefinite constraints removed to give NLP

    Args:
        model: empty gurobi model object
        OB_bounds: confidence intervals on observed moments up to order d
        beta: capture efficiency vector
        d: maximum moment order used
        U: unobserved species indices
        time_limit: optimization time limit

    Returns:
        model: gurobi model object with NLP constraints (all but semidefinite)
        variables: dict for model variable reference
    '''

    # model settings
    model.Params.TimeLimit = time_limit
    K = 100

    # helpful values
    R = 4
    S = 2
    Nd = compute_Nd(d)
    moment_powers = compute_powers(d, S)

    # variables
    y = model.addMVar(shape=Nd, vtype=GRB.CONTINUOUS, name="y", lb=0)
    k = model.addMVar(shape=R, vtype=GRB.CONTINUOUS, name="k", lb=0, ub=K)

    # moment matrices
    M_0 = construct_M_0(y, d, S)
    M_1 = construct_M_s(y, d, 1, S)
    M_2 = construct_M_s(y, d, 2, S)
    
    # constraints

    # confidence interval bounds on OB moments
    y_lb = OB_bounds[0, :]
    y_ub = OB_bounds[1, :]

    # B scaling matrix
    B = compute_B(beta, d, U, S)

    # moment bounds
    model.addConstr(B @ y <= y_ub, name="y_UB")
    model.addConstr(B @ y >= y_lb, name="y_LB")

    # moment equations
    for alpha in moment_powers:
        A_alpha_d = compute_A(alpha, d)
        model.addConstr(k.T @ A_alpha_d @ y == 0, name=f"ME_{alpha}_{d}")

    # telegraph moment equality (as G in {0, 1}, E[G^n] = E[G] for n > 0, same with cross moments)
    # NOTE: order(alpha) <= d, just coincides here with moment equation powers
    for i, alpha in enumerate(moment_powers):
        # if moment includes G^n for n > 0, then equal to moment with just G
        if alpha[1] > 0:
            j = moment_powers.index([alpha[0], 1])
            model.addConstr(y[i] == y[j], name="Telegraph_moment_equality")

    # base
    model.addConstr(y[0] == 1, name="y0_base")

    # fixed parameter
    model.addConstr(k[3] == 1, name="k4_fixed")

    # variable dict
    variables = {
        'y': y,
        'k': k,
        'M_0': M_0,
        'M_1': M_1,
        'M_2': M_2
    }

    return model, variables

In [15]:
def optimize(model):
    '''Optimize model with no objective, return status.'''

    # optimize
    model.setObjective(0, GRB.MINIMIZE)
    model.optimize()
    status = status_codes[model.status]

    return model, status

In [48]:
def semidefinite_cut(model, variables, print_evals=False):
    '''
    Check semidefinite feasibility of NLP feasible point
    Feasible: stop
    Infeasible: add cutting plane

    Args:
        model: optimized NLP model
        variables: model variable reference dict
        print_evals: option to display moment matrix eigenvalues (semidefinite condition)

    Returns:
        model: model with any cutting planes added
        bool: semidefinite feasibility status
    '''

    # get moment matrix values
    M_0_val = variables['M_0'].X
    M_1_val = variables['M_1'].X
    M_2_val = variables['M_2'].X

    # check semidefinite
    evals_0, evecs_0 = np.linalg.eigh(M_0_val)
    evals_1, evecs_1 = np.linalg.eigh(M_1_val)
    evals_2, evecs_2 = np.linalg.eigh(M_2_val)

    if print_evals:
        print("Moment matices eigenvalues:")
        print(evals_0)
        print(evals_1)
        print(evals_2)

    # positive eigenvalues
    if (evals_0 >= 0).all() and (evals_1 >= 0).all() and (evals_2 >= 0).all():

        print("SDP feasible\n")
    
        return model, True

    # negative eigenvalue
    else:

        print("SDP infeasible")

        if evals_0[0] < 0:

            # get eigenvector
            v = evecs_0[:, 0]

            # add cutting plane
            model.addConstr(v.T @ variables['M_0'] @ v >= 0, name="Cut_0")

            print("M_0 cut added\n")

        if evals_1[0] < 0:

            # get eigenvector
            v = evecs_1[:, 0]

            # add cutting plane
            model.addConstr(v.T @ variables['M_1'] @ v >= 0, name="Cut_1")

            print("M_1 cut added\n")

        if evals_2[0] < 0:

            # get eigenvector
            v = evecs_2[:, 0]

            # add cutting plane
            model.addConstr(v.T @ variables['M_2'] @ v >= 0, name="Cut_2")

            print("M_2 cut added\n")

    return model, False

In [49]:
def semidefinite_cut_all(model, variables, print_evals=False, eval_eps=10**-6):
    '''
    Check semidefinite feasibility of NLP feasible point
    Feasible: stop
    Infeasible: add cutting plane (ALL negative eigenvalues)

    Args:
        model: optimized NLP model
        variables: model variable reference dict
        print_evals: option to display moment matrix eigenvalues (semidefinite condition)

    Returns:
        model: model with any cutting planes added
        bool: semidefinite feasibility status
    '''

    # get moment matrix values
    M_0_val = variables['M_0'].X
    M_1_val = variables['M_1'].X
    M_2_val = variables['M_2'].X

    # check semidefinite
    evals_0, evecs_0 = np.linalg.eigh(M_0_val)
    evals_1, evecs_1 = np.linalg.eigh(M_1_val)
    evals_2, evecs_2 = np.linalg.eigh(M_2_val)

    if print_evals:
        print("Moment matices eigenvalues:")
        print(evals_0)
        print(evals_1)
        print(evals_2)

    # positive eigenvalues
    if (evals_0 >= -eval_eps).all() and (evals_1 >= -eval_eps).all() and (evals_2 >= -eval_eps).all(): 

        print("SDP feasible\n")
    
        return model, True

    # negative eigenvalue
    else:

        print("SDP infeasible\n")

        # for each M_0 eigenvalue
        for i, lam in enumerate(evals_0):

            # if negative (sufficiently)
            if lam < -eval_eps:

                # get evector
                v = evecs_0[:, i]

                # add cutting plane
                model.addConstr(v.T @ variables['M_0'] @ v >= 0, name="Cut_0")
            
                print("M_0 cut added")

        # for each M_1 eigenvalue
        for i, lam in enumerate(evals_1):

            # if negative (sufficiently)
            if lam < -eval_eps:

                # get evector
                v = evecs_1[:, i]

                # add cutting plane
                model.addConstr(v.T @ variables['M_1'] @ v >= 0, name="Cut_1")

                print("M_1 cut added")

        # for each M_2 eigenvalue
        for i, lam in enumerate(evals_2):

            # if negative (sufficiently)
            if lam < -eval_eps:

                # get evector
                v = evecs_2[:, i]

                # add cutting plane
                model.addConstr(v.T @ variables['M_2'] @ v >= 0, name="Cut_2")

                print("M_2 cut added")

        print("")

    return model, False

In [50]:
def feasibility_test(OB_bounds, beta, d, U, print_evals=False, eval_eps=10**-6):
    '''
    Full feasibility test of birth death model via following algorithm

    Optimize NLP
    Infeasible: stop
    Feasible: check SDP feasibility
        Feasible: stop
        Infeasible: add cutting plane and return to NLP step

    Args:
        OB_bounds: confidence intervals on observed moments up to order d
        beta: capture efficiency vector
        d: maximum moment order used
        U: unobserved species indices

    Returns:
        None
    '''

    # silent
    options = {'OutputFlag': 0}

    # environment context
    with gp.Env(params=options) as env:

        # model context
        with gp.Model('test-SDP', env=env) as model:

            # construct base model (no semidefinite constraints)
            model, variables = base_model(model, OB_bounds, beta, d, U)

            # check feasibility
            model, status = optimize(model)

            # while feasible
            while status == "OPTIMAL":

                print("NLP feasible")

                # check semidefinite feasibility
                model, semidefinite_feas = semidefinite_cut_all(model, variables, print_evals=print_evals, eval_eps=eval_eps)

                # semidefinite feasible
                if semidefinite_feas:
                    break

                # semidefinite infeasible
                else:

                    # check feasibility with added cut
                    model, status = optimize(model)

            # if infeasible
            if status == "INFEASIBLE":
                print("SDP infeasible")

                #model.computeIIS()
                #model.write('test.ilp')

In [51]:
def bootstrap(sample, d, U, S=2, N=1000):
    '''
    Estimate bootstrap intervals of sample moments up to order d

    Args:
        sample: integer sample of length n
        d: maximum moment order to estimate
        N: number of bootstrap samples
        S: number of species

    Returns:
        y_bounds: (2, Nd) array of moment confidence intervals
    '''

    # helpful values
    powers = compute_powers(d, S)
    Nd = compute_Nd(d, S)

    # get sample size
    n = len(sample)

    # bootstrap to N x n
    boot = rng.choice(sample, size=(N, n))

    # estimate
    y_bounds = np.zeros((2, Nd))
    for i, alpha in enumerate(powers):

        # check if unobserved species present in moment
        unobserved_moment = False
        for j, alpha_j in enumerate(alpha):
            if (j in U) and (alpha_j > 0):
                unobserved_moment = True

        # unobsreved: [0, inf] bounds
        if unobserved_moment:
            y_bounds[:, i] = np.array([0, np.inf])

        # otherwise estimate
        else:

            # mean over sample
            moment_estimates = np.mean(boot ** alpha[0], axis=1)

            # quantile over boot
            moment_interval = np.quantile(moment_estimates, [0.025, 0.975])

            # store
            y_bounds[:, i] = moment_interval

    return y_bounds

In [52]:
def downsample_data(sample, b):

    n = len(sample)

    # capture efficiency
    if b == 0:
        beta = np.ones(n)
    else:
        beta = rng.beta(1, b, size=1000)

    # downsample
    downsample = rng.binomial(sample, beta).tolist()

    return downsample, beta

## Interaction test

In [53]:
from interaction_inference.simulation import gillespie_telegraph

In [54]:
# settings
k_on = 1
k_off = 0
k1 = 2
k2 = 1
k_reg = 5
b = 0
n = 1000
N = 1000

# sample
params = {
    'k_on_1': k_on,
    'k_on_2': k_on,
    'k_off_1': k_off,
    'k_off_2': k_off,
    'k_tx_1': k1,
    'k_tx_2': k1,
    'k_deg_1': k2,
    'k_deg_2': k2,
    'k_reg': k_reg
}
sample = gillespie_telegraph(params, n)
sample = [x[0] for x in sample]

# downsample
downsample, beta = downsample_data(sample, b)

downsample = rng.choice([0, 1, 5, 6, 10, 11, 20, 21], size=n).tolist()

# mean expression level
print(f"Mean expression {np.mean(downsample)}")

Mean expression 9.223


In [55]:
# settings
d = 3

# bootstrap
y_bounds = bootstrap(downsample, d, U=[1])

# test feasibility
feasibility_test(y_bounds, beta, d, U=[1], print_evals=True, eval_eps=10**-6)

NLP feasible
Moment matices eigenvalues:
[  0.18039037   0.35888875 137.38890765]
[1.67196340e+00 2.13641253e+00 2.46681178e+03]
[8.61604579e-15 1.41437970e-01 7.91960616e+01]
SDP feasible



## Birth-Death interaction

In [56]:
from interaction_inference.simulation import gillespie_birth_death

In [57]:
# settings
k1 = 5
k2 = 1
k_reg = 5
b = 0
n = 1000
N = 1000

# sample
params = {
    'k_tx_1': k1,
    'k_tx_2': k1,
    'k_deg_1': k2,
    'k_deg_2': k2,
    'k_reg': k_reg
}
sample = gillespie_birth_death(params, n)
sample = [x[0] for x in sample]

# downsample
downsample, beta = downsample_data(sample, b)

# mean expression level
print(f"Mean expression {np.mean(downsample)}")

Mean expression 1.292


In [58]:
# settings
d = 7

# bootstrap
y_bounds = bootstrap(downsample, d, U=[1])

# test feasibility
feasibility_test(y_bounds, beta, d, U=[1], print_evals=True, eval_eps=10**-6)

NLP feasible
Moment matices eigenvalues:
[-8.54846117e-16  4.08657310e-17  1.55871305e-15  7.65565149e-02
  1.19455250e-01  4.32215938e-01  9.58629635e-01  7.63281196e+00
  1.95767692e+01  2.32425105e+03]
[-8.79338179e-15 -1.60553178e-16  1.03623072e-15  1.94882073e-02
  5.39391549e-02  5.79632925e-01  1.66502939e+00  3.32776241e+01
  9.52297065e+01  1.55510246e+04]
[-3.56943148e-15 -3.74555818e-16 -1.85077130e-17  9.78165015e-16
  1.44544134e-13  4.03583827e-13  9.78795045e-02  7.64238114e-01
  1.23772804e+01  1.74213666e+03]
SDP feasible



## Notes

- added threshold for evalues < 0 (as was stuck on -10**16 values etc)

- fixed missing M_2 moment matrix